# ICEES Asthma, PM2.5 Exposure, and Drug Efficacy

In [1]:
import networkx
import requests
import json
requests.packages.urllib3.disable_warnings()

In [2]:
tabular_headers = {"Content-Type" : "application/json", "accept": "text/tabular"}
json_headers = {"Content-Type" : "application/json", "accept": "application/json"}

## COHORT:44 (N = 1601) from feature variable AvgDailyPM2.5Exposure < 3

In [3]:
cohort_44_definition = '{"AvgDailyPM2.5Exposure": { "value": 3, "operator": "<" }}'
cohort_44_response = requests.post('https://icees.renci.org/1.0.0/patient/2010/cohort', data=cohort_44_definition, headers = json_headers, verify = False)
cohort_44_json = cohort_44_response.json()
print(cohort_44_json['return value'])

{'size': 1601, 'cohort_id': 'COHORT:44'}


## COHORT:54 (N = 3408) from feature variable AvgDailyPM2.5Exposure > 3

In [4]:
cohort_54_definition = '{"AvgDailyPM2.5Exposure": { "value": 3, "operator": ">" }}'
cohort_54_response = requests.post('https://icees.renci.org/1.0.0/patient/2010/cohort', data=cohort_54_definition, headers = json_headers, verify = False)
cohort_54_json = cohort_54_response.json()
print(cohort_54_json['return value'])

{'size': 3408, 'cohort_id': 'COHORT:54'}


## TotalEDInpatientVisits < 2, /associations_to_all_features endpoint, cohort 44

In [5]:
all_features_call = '{"feature":{"TotalEDInpatientVisits":{"operator":"<", "value":2}},"maximum_p_value":0.1}'
data_for_all_cohorts = {}
cohort_44_response = requests.post('https://icees.renci.org/1.0.0/patient/2010/cohort/COHORT%3A44/associations_to_all_features', headers = json_headers, data=all_features_call, verify = False)
cohort_44_response_json = cohort_44_response.json()
cohort_44_common_medication_data = []

In [20]:
cohort_44_return_value = cohort_44_response_json['return value']
for section in cohort_44_return_value:
    feature_name = section['feature_b']['feature_name']
    cohort_44_common_medication_data.append(section)
data_for_all_cohorts['COHORT:44']=cohort_44_common_medication_data

## TotalEDInpatientVisits < 2, /associations_to_all_features endpoint, cohort 54

In [7]:
cohort_54_response = requests.post('https://icees.renci.org/1.0.0/patient/2010/cohort/COHORT%3A54/associations_to_all_features', headers = json_headers, data=all_features_call, verify = False)
cohort_54_response_json = cohort_54_response.json()
cohort_54_common_medication_data = []

In [8]:
cohort_54_return_value = cohort_54_response_json['return value']
for section in cohort_54_return_value:
    feature_name = section['feature_b']['feature_name']
    cohort_54_common_medication_data.append(section)
data_for_all_cohorts['COHORT:54']=cohort_54_common_medication_data

## With the 'associations_to_all_features' data in hand for cohorts 44 and 54, a dictionary containing quantities calculated from these cohorts is produced.

In [66]:
from collections import defaultdict
all_cohorts_dict = defaultdict(dict)
not_drugs = ['AgeStudyStart', 'CoughDx', 'PneumoniaDx','AvgDailyOzoneExposure','ObesityICD','AsthmaDx', 'EstProbabilityHighSchoolMaxEducation']

for cohort_id, cohort_content in data_for_all_cohorts.items():
    cohort_dict = defaultdict(dict)
    number_patients_in_cohort = cohort_content[0]['total']
    patients_with_ED_under_2_total_cohort = cohort_content[0]['columns'][0]['frequency']
    patients_with_ED_over_2_total_cohort = cohort_content[0]['columns'][1]['frequency']
    percentage_patients_with_ED_under_2_total_cohort = 100*patients_with_ED_under_2_total_cohort/number_patients_in_cohort
    for drug in cohort_content:
        drug_name = drug['feature_b']['feature_name']
        if drug_name not in not_drugs:
            drug_dict = defaultdict(dict)
            number_patients_this_drug_EDVisits_under_2 = drug['feature_matrix'][1][0]['frequency']
            number_patients_this_drug_EDVisits_over_2 = drug['feature_matrix'][1][1]['frequency']
            number_patients_admin_this_drug = drug['rows'][1]['frequency']
            number_patients_other_drugs_EDVisits_less_than_2 = drug['feature_matrix'][0][0]['frequency']
            number_patients_other_drugs_EDVisits_over_2 = drug['feature_matrix'][0][1]['frequency']
            number_patients_admin_other_drugs = drug['rows'][0]['frequency']
            Prob_A_given_B = 100*number_patients_this_drug_EDVisits_under_2/number_patients_admin_this_drug
            drug_dict['For all patients taking Drug, the percentage with EDVisits<2'] = Prob_A_given_B
            Prob_B_given_A = 100*(number_patients_this_drug_EDVisits_under_2/patients_with_ED_under_2_total_cohort)
            drug_dict['For all patients with EDVisits<2, the percentage taking Drug'] = Prob_B_given_A
            drug_dict['% of patients on Drug w/ EDVisits<2 - % of patients in cohort w/ EDVisits<2'] = round(Prob_A_given_B - percentage_patients_with_ED_under_2_total_cohort, 2)
            cohort_dict[drug_name] = dict(drug_dict)
    all_cohorts_dict[cohort_id+' responsiveness']=dict(cohort_dict)

In [67]:
import pandas

all_cohorts_dict_reformatted = {(outerKey, innerKey): values for outerKey, innerDict in all_cohorts_dict.items() for innerKey, values in innerDict.items()}
all_cohorts_dict_table = pandas.DataFrame(all_cohorts_dict_reformatted)
all_cohorts_dict_table.round(2)

COHORT:44 responsiveness  \
                                                                 Prednisone   
% of patients on Drug w/ EDVisits<2 - % of pati...                    -7.61   
For all patients taking Drug, the percentage wi...                    84.46   
For all patients with EDVisits<2, the percentag...                    11.06   

                                                                    \
                                                   Diphenhydramine   
% of patients on Drug w/ EDVisits<2 - % of pati...           -7.64   
For all patients taking Drug, the percentage wi...           84.43   
For all patients with EDVisits<2, the percentag...           13.98   

                                                   COHORT:54 responsiveness  \
                                                                 Prednisone   
% of patients on Drug w/ EDVisits<2 - % of pati...                    -8.46   
For all patients taking Drug, the percentage wi...                    70.24   
For all patients with EDVisits<2, the percentag...                     9.77   

                                                                    \
                                                   Diphenhydramine   
% of patients on Drug w/ EDVisits<2 - % of pati...          -12.03   
For all patients taking Drug, the percentage wi...           66.67   
For all patients with EDVisits<2, the percentag...           11.26   

                                                                           
                                                   Ipratropium Salmeterol  
% of patients on Drug w/ EDVisits<2 - % of pati...      -17.30     -45.36  
For all patients taking Drug, the percentage wi...       61.40      33.33  
For all patients with EDVisits<2, the percentag...        4.92       0.11